In [17]:
import numpy as np

def are_likely_isomorphic(graph1, graph2):
    """
    Determine if two graphs are likely to be isomorphic based on four criteria:
    a. Equal number of vertices
    b. Equal number of edges
    c. Same degree sequences
    d. Same sorted list of lists of degrees of adjacent vertices
    
    Parameters:
    graph1, graph2: numpy adjacency matrices
    
    Returns:
    dictionary with results of each criteria
    """
    results = {}
    
    # a. Equal number of vertices
    n1 = graph1.shape[0]
    n2 = graph2.shape[0]
    results['equal_vertices'] = (n1 == n2)
    
    # b. Equal number of edges
    edges1 = np.sum(graph1) / 2  # Divide by 2 since each edge is counted twice
    edges2 = np.sum(graph2) / 2
    results['equal_edges'] = (edges1 == edges2)
    
    # If different number of vertices, we can't compare the following criteria
    if not results['equal_vertices']:
        results['same_degree_sequence'] = False
        results['same_neighbor_degree_lists'] = False
        return results
    
    # c. Same degree sequences
    degrees1 = np.sum(graph1, axis=1)
    degrees2 = np.sum(graph2, axis=1)
    
    sorted_degrees1 = np.sort(degrees1)[::-1]  # Sort in descending order
    sorted_degrees2 = np.sort(degrees2)[::-1]
    
    results['same_degree_sequence'] = np.array_equal(sorted_degrees1, sorted_degrees2)
    
    # d. Same sorted lists of degrees of adjacent vertices
    neighbor_degrees1 = []
    neighbor_degrees2 = []
    
    for i in range(n1):
        # Get neighbors of vertex i
        neighbors1 = np.where(graph1[i] > 0)[0]
        # Get degrees of those neighbors
        neighbor_deg1 = degrees1[neighbors1]
        # Sort and add to list
        neighbor_degrees1.append(sorted(neighbor_deg1.tolist()))
        
        # Do the same for graph2
        neighbors2 = np.where(graph2[i] > 0)[0]
        neighbor_deg2 = degrees2[neighbors2]
        neighbor_degrees2.append(sorted(neighbor_deg2.tolist()))
    
    # Sort the lists of neighbor degrees 
    neighbor_degrees1.sort()
    neighbor_degrees2.sort()
    
    results['same_neighbor_degree_lists'] = (neighbor_degrees1 == neighbor_degrees2)
    
    # result
    results['likely_isomorphic'] = all(results.values())
    
    return results

def is_bipartite(graph):
    """
    Determine if a graph is bipartite using BFS coloring.
    
    Parameters:
    graph: numpy adjacency matrix (2D numpy array)
    
    Returns:
    is_bipartite: boolean indicating if the graph is bipartite
    partition: list where each element is 0 or 1 indicating the partition
               (-1 if the vertex hasn't been assigned or if graph is not bipartite)
    """
    n = graph.shape[0]
    colors = np.full(n, -1)  # -1: uncolored, 0: set A, 1: set B
    
    # Function to check bipartiteness starting from a specific vertex
    def bfs_coloring(start):
        queue = [start]
        colors[start] = 0  # Assign first color
        
        while queue:
            current = queue.pop(0)
            current_color = colors[current]
            next_color = 1 - current_color  # Flip between 0 and 1
            
            # Find all neighbors
            neighbors = np.where(graph[current] > 0)[0]
            
            for neighbor in neighbors:
                if colors[neighbor] == -1:  # Uncolored
                    colors[neighbor] = next_color
                    queue.append(neighbor)
                elif colors[neighbor] == current_color:  # Same color conflict
                    return False
        
        return True
    
    # We need to run BFS from every unvisited vertex to handle disconnected graphs
    for i in range(n):
        if colors[i] == -1:  # If vertex hasn't been colored yet
            if not bfs_coloring(i):
                return False, colors
    
    return True, colors

def get_bipartition(graph):
    """
    If the graph is bipartite, return the two sets of vertices.
    
    Parameters:
    graph: numpy adjacency matrix
    
    Returns:
    is_bipartite: boolean indicating if the graph is bipartite
    partition_a: list of vertices in set A
    partition_b: list of vertices in set B
    """
    is_bip, colors = is_bipartite(graph)
    
    if not is_bip:
        return False, [], []
    
    partition_a = np.where(colors == 0)[0].tolist()
    partition_b = np.where(colors == 1)[0].tolist()
    
    return True, partition_a, partition_b

def verify_bipartition(graph, partition_a, partition_b):
    """Verify that a bipartition is valid"""
    valid = True
    # Check no edges within partition A
    for i in partition_a:
        for j in partition_a:
            if i != j and graph[i, j] > 0:
                valid = False
                print(f"Invalid partition! Edge within Partition A: {i}-{j}")
    
    # Check no edges within partition B
    for i in partition_b:
        for j in partition_b:
            if i != j and graph[i, j] > 0:
                valid = False
                print(f"Invalid partition! Edge within Partition B: {i}-{j}")
    
    if valid:
        print("Partitioning verified successfully!")
    else:
        print("Partitioning verification failed!")

# Test part 1 with a example 
g1 = np.array([
    [0, 1, 0, 0],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 0]
])

g2 = np.array([
    [0, 1, 1, 1],
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 0]
])

print("Testing if graphs are likely isomorphic:")
results = are_likely_isomorphic(g1, g2)
for criterion, result in results.items():
    print(f"{criterion}: {result}")

# Test part 2 with a bipartite graph
g_even_cycle = np.array([
    [0, 1, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [1, 0, 1, 0]
])

print("\nTesting bipartite functionality on even cycle:")
is_bip, partition_a, partition_b = get_bipartition(g_even_cycle)
print(f"Is bipartite: {is_bip}")
print(f"Partition A: {partition_a}")
print(f"Partition B: {partition_b}")
verify_bipartition(g_even_cycle, partition_a, partition_b)

Testing if graphs are likely isomorphic:
equal_vertices: True
equal_edges: True
same_degree_sequence: False
same_neighbor_degree_lists: False
likely_isomorphic: False

Testing bipartite functionality on even cycle:
Is bipartite: True
Partition A: [0, 2]
Partition B: [1, 3]
Partitioning verified successfully!
